In [33]:
import os
from dotenv import load_dotenv

from semantic_kernel.kernel import Kernel
from semantic_kernel.functions.kernel_function import KernelFunction
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import AzureChatPromptExecutionSettings

In [34]:
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings

In [35]:
# Load environment variables
load_dotenv()

# Create the kernel
kernel = Kernel()

# Add Azure OpenAI chat completion service
chat_completion = AzureChatCompletion(
    deployment_name="gpt-4o-mini",  # Replace with your actual deployment name
    endpoint="https://exquitech-openai-2.openai.azure.com/",
    api_key=os.getenv("AZURE_OPENAI_KEY"),
)
kernel.add_service(chat_completion, "chat_completion")

In [36]:
# Define the prompt
prompt = "Give a fun fact about: {{$input}}"

This is your prompt template string.

{{$input}} is a placeholder that will be replaced dynamically with the user input (like "bananas").

In [37]:
# Execution settings for Azure
execution_settings = AzureChatPromptExecutionSettings(
    service_id="default",  # match what you set when adding the service
    ai_model_id="gpt-4o-mini",  # or whatever your Azure deployment name is
    temperature=0.7,
)


This configures how the prompt will be sent to Azure:

- temperature=0.7: Controls randomness in generation (higher = more creative).

- ai_model_id: The Azure model to use.

- service_id: Connects to the previously added service named "default" or "chat_completion".

In [38]:
# Prompt template config
prompt_template_config = PromptTemplateConfig(
    name="GetFunFact",
    template=prompt,
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="Topic to get a fun fact about", is_required=True)
    ],
    execution_settings=execution_settings,
)

This creates a structured prompt template object.

InputVariable defines what input the prompt expects (input variable with description).

This formalizes how inputs will be injected into the prompt string.

execution_settings ties the prompt to how it will be executed by the AI.

In [ ]:
fun_fact_function = kernel.add_function(
    prompt=prompt,
    plugin_name="FunFacts",                         # Logical grouping of related functions
    function_name="GetFunFact",                     # Function identifier inside the plugin
    prompt_execution_settings=execution_settings,   # Execution settings for this function
)


This creates a KernelFunction from your prompt string:

- This function knows how to:

    - Take input,

    - Inject it into the prompt,

    - Send the prompt to the AI service,

    - Return the AI response.

plugin_name lets you group similar functions together (like "FunFacts").

function_name names this specific function for invocation.

In [44]:
result = await kernel.invoke(plugin_name="FunFacts", function_name="GetFunFact", input="bananas")
print(str(result))


A fun fact about bananas is that they are technically classified as berries! In botanical terms, a berry is a fruit produced from the ovary of a single flower with seeds embedded in the flesh. So, while we often think of strawberries and raspberries as berries, bananas fit the true definition of a berry!


You call the function, passing "bananas" as the input.

Kernel will:

- Inject "bananas" into the prompt: "Give a fun fact about: bananas".

- Send prompt to Azure OpenAI.

- Wait for the AI's response.

- Return the response as result.

You print the fun fact response.